# Naive RAG

The following codes are the implementation of Naive RAG.

In [2]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_unstructured import UnstructuredLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata

In [3]:
load_dotenv()
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

True

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini")

In [5]:
file_paths = [
    "/Users/krimssmirk/Desktop/rag-llm/document.txt"
]

# Load, chunk and index the contents of the blog.
loader = UnstructuredLoader(file_paths)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=filter_complex_metadata(splits), embedding=HuggingFaceEmbeddings())

/var/folders/0r/dktk9n0551j0gkccz4k6246h0000gn/T/ipykernel_12525/4250621703.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  vectorstore = Chroma.from_documents(documents=filter_complex_metadata(splits), embedding=HuggingFaceEmbeddings())
/var/folders/0r/dktk9n0551j0gkccz4k6246h0000gn/T/ipykernel_12525/4250621703.py:11: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = Chroma.from_documents(documents=filter_complex_metadata(splits), embedding=HuggingFaceEmbeddings())
/Users/krimssmirk/Des

In [7]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is application form?")

/Users/krimssmirk/Desktop/rag-llm/.venv/lib/python3.11/site-packages/langsmith/client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


"An application form is a document used to collect necessary information from individuals applying for a specific purpose, such as a visa. It typically requires personal details like nationality, occupation, and relationships, as well as additional documentation based on the applicant's circumstances. This ensures that all required information is gathered for processing the application."